In [ ]:
from google.colab import drive
import sqlite3
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [ ]:
drive.mount('/content/drive')

!ls "/content/drive/MyDrive/Data 270 project/Data/"

In [ ]:
data =pd.read_csv('/content/drive/MyDrive/Data 270 project/Data/NORMALIZED_TABLE.csv')
df = pd.DataFrame(data)


In [ ]:
date = df["DATE"]

In [ ]:
df = df.drop(columns=['DATE'])

In [ ]:
df.columns

In [ ]:
features=(df.columns[:-1])
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['storage']].values
x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3','principal component 4','principal component 5','principal component 6','principal component 7','principal component 8'])

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
finalDf = pd.concat([principalDf, df[['storage']]], axis = 1)

In [ ]:
finalDf

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
X = df.iloc[:,0:(len(df.columns)-1)].values
y = df.iloc[:,-1].values
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
df

In [ ]:
X

In [ ]:
'''import xgboost as xgb
from sklearn.metrics import mean_squared_error
X = finalDf.iloc[:,0:(len(finalDf.columns)-1)].values
y = finalDf.iloc[:,-1].values
data_dmatrix = xgb.DMatrix(data=X,label=y)'''

In [ ]:
import random 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import RepeatedKFold
# set aside 20% of train and test data for evaluation
random.seed(13)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True)

print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(n_estimators=100,max_depth=4,learning_rate=0.1,subsample=0.5, verbosity=0)
xg_reg.fit(X_train,y_train)

#xg_reg = xgb.XGBRegressor( colsample_bytree=0.5, max_depth=3, learning_rate=0.1, n_estimators=500, subsample=0.5, objective='reg:linear', booster='gbtree',verbosity=0)
#gamma = 5,subsample=0.5,learning_rate=0.1,max_depth=3,reg_alpha=1, reg_lambda=1,

#R2 or coefficient of determination
train_score = xg_reg.score(X_train,y_train)  
print("Training score: ", train_score)

#validation
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xg_reg, X_train, y_train, cv=kfold )
print("K-fold CV average training score: " + str(kf_cv_scores.mean()))

#validation
kfold_repeat = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
kf_repeat_cv_scores = cross_val_score(xg_reg, X_train, y_train, cv=kfold_repeat )
print("K-fold Repeating average training score:" + str(kf_repeat_cv_scores.mean()))


preds = xg_reg.predict(X_test)
train_score1 = xg_reg.score(X_test,y_test)  
print("Testing score: ", train_score1)


#validation
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xg_reg, X_test, y_test, cv=kfold )
print("K-fold CV average testing score: " + str(kf_cv_scores.mean()))

#validation
kfold_repeat = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
kf_repeat_cv_scores = cross_val_score(xg_reg, X_test, y_test, cv=kfold_repeat )
print("K-fold Repeating average testing score:" + str(kf_repeat_cv_scores.mean()))


mse = mean_squared_error(y_test, preds)
print("MSE: %.2f" % mse)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, preds, label="predicted")
plt.title("Santa Clara water supply test and predicted data")
plt.legend()
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance
perm_importance = permutation_importance(xg_reg, X_test, y_test)
sorted_idx = perm_importance.importances_mean.argsort()

feature_names = np.array(list(df)[0:len(list(df))-1])

In [ ]:
perm_importance1 = permutation_importance(xg_reg, X_train, y_train)
sorted_idx1 = perm_importance1.importances_mean.argsort()

feature_names1 = np.array(list(df)[0:len(list(df))-1])

In [ ]:
plt.figure(figsize = (8,5))
plt.barh(feature_names[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

In [ ]:
plt.figure(figsize = (8,5))
plt.barh(feature_names1[sorted_idx], perm_importance1.importances_mean[sorted_idx1])
plt.xlabel("Permutation Importance")